<a href="https://colab.research.google.com/github/purang2/BERT/blob/main/lossfunc_TEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install

In [1]:
!pip install transformers
!pip install datasets #from huggingface
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 32.2 MB/s 
     |████████████████████████████████| 6.6 MB 67.6 MB/s 
     |████████████████████████████████| 163 kB 93.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 432 kB 31.9 MB/s 
     |████████████████████████████████| 212 kB 78.9 MB/s 
     |████████████████████████████████| 115 kB 70.2 MB/s 
     |████████████████████████████████| 127 kB 71.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 8.4 MB/s 


## Data Preprocessing

In [2]:
from datasets import load_dataset
import pandas as pd 

data_path ="drive/MyDrive/호반우NLP_한국인감정/KERC22Dataset/KERC22Dataset_PublicTest"


train_data = pd.read_csv(data_path+"/train_data.tsv", sep='\t')
train_labels = pd.read_csv(data_path+"/train_labels.csv")

train_data

,sentence_id,person,sentence,scene,context
0,1,어영,야! 전화 받아. 아무리 바빠도 내전화는 받아야 되는거 아냐? 약속 하나도 못지키는...,S0104,NaN
1,2,어영,우리 아무래도 안되겠다. 이게 최선인거 같애. 평생 잊지 않을게. 행복하길 바란다.,S0108,"포기한듯 탁 일어서는데, 띵동 문자. 후다닥 보는 어영. 기막혀 읽어보는"
2,3,지구대,"김경사님, 아직 안가셨어요? 시간 다됐을텐데.",S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대
3,4,순경,근무중인데 어딜가?,S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대
4,5,지구대,다녀오세요. 이런날은 무조건 가서 축하해주셔야죠. 이순경이 대신 근무선다고 나온대요.,S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대
...,...,...,...,...,...
7334,12285,청난,종남이 맛있는것도 사주고 잘 챙겨줘.,S3576,NaN
7335,12286,건강,걱정마라. 먹어도 같이 먹고 굶어도 같이 굶는다.,S3576,NaN
7336,12287,종남,아빠. 우리 저거 타.,S3576,NaN
7337,12288,건강,"나 못타는데. 한번도 안타봤어. 툭하면 차멀미까지 하는데, 봐라, 너도 내몸땡이에 ...",S3576,NaN


In [3]:
train_drop = train_data[['person', 'sentence', 'context']]
train_drop['label'] = train_labels['label']
train_pair = train_drop


import numpy as np 

train_pair['context2'] = train_pair['context'].fillna('none')
train_pair['sentence2'] = train_pair['sentence'].fillna('none')

train_pair


train_pair = train_pair[['person', 'context2', 'sentence2', 'label']]
train_pair['scene'] = train_data['scene']
train_pair



train = list()
scene_df = train_pair['scene']
scene_df = scene_df.drop_duplicates().reset_index(drop=True)


scene_df


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


0      S0104
1      S0108
2      S0109
3      S0115
4      S0119
       ...  
902    S3571
903    S3573
904    S3574
905    S3575
906    S3576
Name: scene, Length: 907, dtype: object

In [4]:

context_df = train_pair['context2']
context_df = context_df.drop_duplicates().reset_index(drop=True)


context_df


0                                                   none
1               포기한듯 탁 일어서는데, 띵동 문자. 후다닥 보는 어영. 기막혀 읽어보는
2        초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대
3      경찰들에 둘러싸여 축하받는 이상. 사람들도 사진찍고 시끌벅적. 좀 떨어져 그 모습보...
4      장바구니 든채 우악스레 애들 양손잡아 끌고가는 도우미. 엎어질듯 질질끌려가는 애들....
                             ...                        
721                           에버랜드. 걸어오는 건강종남. 종남 눈 휘둥그레
722      솜사탕 먹으며 신난 종남. 건강을 슬쩍 올려다보고 건강손을 잡는 종남. 느끼는 건강.
723    종남손 꼭 잡고 가는데, 아이를 어깨에 올리고 가는 남자, 부럽게 돌아보는 종남. ...
724                              이삿짐 옮기는 인부들. 열심히 거드는 청난
725    시간경과. 놀이기구 타는 건강종남. 사람들 비명. 신난 종남 소리지르고. 건강 하얗...
Name: context2, Length: 726, dtype: object

In [5]:
scene_df[:]

0      S0104
1      S0108
2      S0109
3      S0115
4      S0119
       ...  
902    S3571
903    S3573
904    S3574
905    S3575
906    S3576
Name: scene, Length: 907, dtype: object

In [6]:
train_drop = train_data[['person', 'sentence', 'context']]
train_drop['label'] = train_labels['label']
train_pair = train_drop

import numpy as np 

train_pair['context2'] = train_pair['context'].fillna('none')
train_pair['sentence2'] = train_pair['sentence'].fillna('none')

train_pair


train_pair = train_pair[['person', 'context2', 'sentence2', 'label']]
train_pair['scene'] = train_data['scene']
train_pair

train = list()
scene_df = train_pair['scene']
scene_df = scene_df.drop_duplicates().reset_index(drop=True)


def mk_scenario(i, df):
  scenario = df.iloc[0]['context2']+" <sep> " 
  if(i==0):
    scenario = scenario + df.iloc[i]['person'] + "가 말하길 " +"\""+ df.iloc[i]['sentence2'] +"\""
    return scenario
  elif(0<i and i<len(df)-1 ):
    scenario = scenario + df.iloc[i-1]['person'] + "가 말하길 " +"\""+ df.iloc[i-1]['sentence2']+"\""+ " " + df.iloc[i]['person'] + "가 말하길 " +"\""+df.iloc[i]['sentence2'] +"\""+" "+df.iloc[i+1]['person'] + "가 말하길 " +"\""+ df.iloc[i+1]['sentence2']+"\""
    return scenario
  elif(i == len(df)-1):
    scenario = scenario + df.iloc[i-1]['person'] + "가 말하길 " +"\""+ df.iloc[i-1]['sentence2'] +"\""+" "+ df.iloc[i]['person'] + "가 말하길 " +"\""+ df.iloc[i]['sentence2'] +"\""
    return scenario


def trans_df(train_pair, train):
  for i in scene_df[:]:
    temp = train_pair[train_pair['scene']==i].reset_index(drop=True)
    train+=[mk_scenario(j, temp) for j in range(len(temp))]
    
trans_df(train_pair, train)
train = pd.DataFrame(train, columns=('scenario',))
print(train.iloc[1]['scenario'])

train['label'] = train_pair['label']
train
train_pairs = train

labels = {'euphoria':0,
          'neutral':1,
          'dysphoria':2
          }

train_pairs['label'] = [labels[label] for label in train_pairs['label']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


포기한듯 탁 일어서는데, 띵동 문자. 후다닥 보는 어영. 기막혀 읽어보는 <sep> 어영가 말하길 "우리 아무래도 안되겠다. 이게 최선인거 같애. 평생 잊지 않을게. 행복하길 바란다."


In [7]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(train_pairs, test_size=0.2, random_state=42)


from datasets import Dataset
import pandas as pd 


train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)


from transformers import AutoTokenizer

max_input_length = 200
#tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-discriminator")
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

def preprocess_function(examples):
    inputs = [doc for doc in examples["scenario"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,padding='max_length')

    return model_inputs

#.remove_columns(["input_texts","target_texts","token_type_ids"])

train_dataset =train_dataset.map(preprocess_function, batched=True).remove_columns(["__index_level_0__",'scenario'])
val_dataset = val_dataset.map(preprocess_function, batched=True).remove_columns(["__index_level_0__",'scenario'])

train_dataset


train_dataset

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/752k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5871
})

## Modeling

In [8]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from datasets import Dataset
import pandas as pd 
from transformers import AutoTokenizer,AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
import evaluate
import torch

max_input_length = 200
#tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-discriminator")
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_preds):
    metric = evaluate.load("f1")
    #metric = evaluate.load('glue', 'qnli')
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    #return metric.compute(predictions=predictions, references=labels)
    return metric.compute(predictions=predictions, references=labels, average='micro')

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
#model = AutoModelForSequenceClassification.from_pretrained("monologg/koelectra-base-discriminator", num_labels=3)
model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-large", num_labels=3)
#klue/roberta-large



from torch import nn
from transformers import Trainer


class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 3.0])).cuda()
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

Downloading:   0%|          | 0.00/547 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'cla

## Train

In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
#model = AutoModelForSequenceClassification.from_pretrained("monologg/koelectra-base-discriminator", num_labels=3)
model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-large", num_labels=3)
#klue/roberta-large

#Gather your training arguments~
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    do_train=True,
    do_eval=True,

#    per_device_train_batch_size=32,
#    per_device_eval_batch_size=32,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)



#Collect your model, training args, dataset, data collator, tokenizer in Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = train_dataset,
    eval_dataset= val_dataset,
    data_collator=data_collator, 
    tokenizer=tokenizer,  
    compute_metrics = compute_metrics  
)

trainer2 = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset = train_dataset,
    eval_dataset= val_dataset,
    data_collator=data_collator, 
    tokenizer=tokenizer,  
    compute_metrics = compute_metrics  
)
trainer2.train()




Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'cla

Epoch,Training Loss,Validation Loss,F1
1,No log,0.445628,0.754768
2,0.477900,0.464473,0.782016
3,0.275700,0.536908,0.803815
4,0.275700,0.743047,0.818120
5,0.134300,0.824750,0.820163


***** Running Evaluation *****
  Num examples = 1468
  Batch size = 16


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1468
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1468
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1468
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json


TrainOutput(global_step=1835, training_loss=0.2514414202614766, metrics={'train_runtime': 3550.2798, 'train_samples_per_second': 8.268, 'train_steps_per_second': 0.517, 'total_flos': 1.0686303751374e+16, 'train_loss': 0.2514414202614766, 'epoch': 5.0})

In [10]:

import gc
gc.collect()
torch.cuda.empty_cache()

## Evaluate

In [13]:

data_path ="drive/MyDrive/호반우NLP_한국인감정/KERC22Dataset/KERC22Dataset_PublicTest"

test_data = pd.read_csv(data_path+"/public_test_data.tsv", sep='\t')

test_pair = test_data[['sentence_id','context','sentence','person', 'scene']]

test_pair['context2'] = test_pair['context'].fillna('none')
test_pair['sentence2'] = test_pair['sentence'].fillna('none')
#모든 글자 합치기
test = list()
scene_df2 = test_pair['scene']
scene_df2 = scene_df2.drop_duplicates().reset_index(drop=True)


def mk_scenario(i, df):
  scenario = df.iloc[0]['context2']+" <sep> " 
  if(i==0):
    scenario = scenario + df.iloc[i]['sentence2']
    return scenario
  elif(0<i and i<len(df)-1 ):
    scenario = scenario + df.iloc[i-1]['sentence2'] + df.iloc[i]['sentence2'] + df.iloc[i+1]['sentence2']
    return scenario
  elif(i == len(df)-1):
    scenario = scenario + df.iloc[i-1]['sentence2'] + df.iloc[i]['sentence2'] 
    return scenario


def trans_df2(test_pair, test):
  for i in scene_df2[:]:
    temp = test_pair[test_pair['scene']==i].reset_index(drop=True)
    test+=[mk_scenario(j, temp) for j in range(len(temp))]
    
trans_df2(test_pair, test)
test = pd.DataFrame(test, columns=('scenario',))
print("--------------------\n", test)

#test_pairs = test_pair[['label','new']]
test['sentence_id'] = test_pair['sentence_id']

test_dataset = Dataset.from_pandas(test)
test_dataset =test_dataset.map(preprocess_function, batched=True).remove_columns(['scenario'])




predictions = trainer.predict(test_dataset)

list1 = []
list2 = []


for i in range(2566):
  list1.append(i)
  list2.append(-1)



raw_data = {'Id': list1,
            'Predicted': list2
            }
df = pd.DataFrame(raw_data)



labels = {'euphoria':0,
          'neutral':1,
          'dysphoria':2
          }

df['Predicted'] = np.argmax(predictions.predictions, axis=-1)

d = ['euphoria','neutral','dysphoria']

df['Predicted'] = [d[label] for label in df['Predicted']]


df['Id'] = test_dataset['sentence_id']

#x = [d[labels[label]] for label in ['euphoria','euphoria','dysphoria','neutral']]

df



df.to_csv('drive/MyDrive/호반우NLP_한국인감정/submission_CV_1001.csv', index=False)

df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


--------------------
                                                scenario
0     음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...
1     음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...
2     음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...
3     음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...
4     음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...
...                                                 ...
2561  가방든 현찰우미. 애들 신났고. 순경건강 나와선. 부러운듯 보는 종남 <sep> 잘...
2562  가방든 현찰우미. 애들 신났고. 순경건강 나와선. 부러운듯 보는 종남 <sep> 다...
2563  가방든 현찰우미. 애들 신났고. 순경건강 나와선. 부러운듯 보는 종남 <sep> 부...
2564  가방든 현찰우미. 애들 신났고. 순경건강 나와선. 부러운듯 보는 종남 <sep> 응...
2565  가방든 현찰우미. 애들 신났고. 순경건강 나와선. 부러운듯 보는 종남 <sep> 그...

[2566 rows x 1 columns]


  0%|          | 0/3 [00:00<?, ?ba/s]

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence_id. If sentence_id are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 2566
  Batch size = 16


,Id,Predicted
0,10,euphoria
1,11,euphoria
2,12,euphoria
3,13,euphoria
4,14,euphoria
...,...,...
2561,12256,euphoria
2562,12257,euphoria
2563,12258,euphoria
2564,12259,euphoria


In [14]:
df['Predicted'].value_counts()

dysphoria    1785
euphoria      564
neutral       217
Name: Predicted, dtype: int64